In [24]:
import pandas as pd
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
import kagglehub
data_dir = 
path = kagglehub.dataset_download("hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images", path=)
print("Path to dataset files:", path)

100%|██████████| 98.7M/98.7M [00:02<00:00, 43.5MB/s]

Extracting files...


Path to dataset files: C:\Users\isali\.cache\kagglehub\datasets\hasnainjaved\melanoma-skin-cancer-dataset-of-10000-images\versions\1


In [18]:
dataset_path = path + '\\melanoma_cancer_dataset\\'

In [20]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.ImageFolder(root=dataset_path+'train\\', transform=transform)
test_dataset = datasets.ImageFolder(root=dataset_path+'test\\', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=2)

images, labels = next(iter(train_loader))

print(images.shape)
print(labels.shape)

torch.Size([32, 3, 64, 64])
torch.Size([32])


In [ ]:
class MelanomaClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super(MelanomaClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        # 64x64 input -> after 3 pooling layers: 64/2/2/2 = 8
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model = MelanomaClassifier()

criterion = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50
batch_size = 32

losses = []

for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        targets = targets.unsqueeze(1).float()
        pred = model(data)
        loss = criterion(pred, targets)

        optim.zero_grad()
        loss.backward()
        optim.step()

    losses.append(loss.item())
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')